In [1]:
from datetime import datetime

In [2]:
class Address:
    def __init__(self, name):
        if type(name).__name__ == "Address":
            name = name.name
        else:
            name = '0x' + name
        self.name = name
        self.address = self
    
    @property
    def str(self):
        return self.name[2:]
    
    def __add__(self, addr):
        if type(addr).__name__ == "Address":
            addr = addr.str
        self.name += addr
        return self
    
    def __iadd__(self, addr):
        self.__add__(addr)
    
    def __repr__(self):
        return f"Address({self.name})"
        
class Eth:
    def __init__(self, amount):
        self.amount = amount
    
    def __add__(self, eth):
        self.amount += eth.amount
        return self
    
    __iadd__ = __add__
    
    def __sub__(self, eth):
        self.amount -= eth.amount
        return self
    
    __isub__ = __sub__
    
    def __neg__(self):
        return -self.amount
    
    def __pos__(self):
        return +self.amount
    
    def __gt__(self, eth):
        return self.amount > eth.amount
    
    def __ge__(self, eth):
        return self.amount >= eth.amount
    
    def __lt__(self, eth):
        return self.amount < eth.amount
    
    def __le__(self, eth):
        return self.amount <= eth.amount
    
    def __eq__(self, eth):
        return self.amount == eth.amount
    
    def __ne__(self, eth):
        return self.amount != eth.amount
    
    def __repr__(self):
        return f"\N{greek capital letter xi}{self.amount}"


class Nft:
    def __init__(self, nftAddr: Address, nftId: int, price: Eth, data: str, contract):
        self.nftAddr = Address(nftAddr)
        self.str = self.nftAddr.str
        self.nftId = nftId
        self.price = price
        self.data = data
        self.contract = contract
    
    @property
    def contractName(self):
        nameLst = list(filter(lambda x: type(x[1]).__name__ == "Address", self.contract.__dict__.items()))
        assert len(nameLst) == 1, f"Could not locate name of contract {self.contract}"
        return nameLst[0][1]
    
    @property
    def contractType(self):
        return type(self.contract).__name__
    
    def __repr__(self):
        return f"Nft({self.nftAddr}, {self.price}, {self.data}, {self.contractName}, {self.contractType})"
    
    def __str__(self):
        return f"Nft({self.nftAddr})"
        
class Owner:
    def __init__(self, address: Address, *args):
        self.address = Address(address)
        self.str = self.address.str
        nfts = set(filter(lambda x: type(x).__name__ == "Nft",args))
        ethSet = set(filter(lambda x: type(x).__name__ == "Eth",args))
        self.eth = Eth(0)
        for ethEl in ethSet:
            self.eth += ethEl
        self.eventLogger = EventLogger()
        for nft in nfts:
            self.setInitialOwner(self.address,nft)
    
    @property
    def assets(self):
        return [self.nfts,self.eth]
    
    @property
    def log(self):
        return self.eventLogger.log
    
    @property
    def nfts(self):
        return Contract.nftsOfAddress(self.address)
    
    def __add__(self, thing):
        if type(thing).__name__ == 'Nft':
            nft = thing
            currOwnerAddr = self.lookupNftOwner(nft)
            if currOwnerAddr:
                self.approve(currOwnerAddr,nft)
                self.transferFrom(currOwnerAddr,self.address,nft)
            else:
                self.setInitialOwner(self.address,nft)
        elif type(thing).__name__ == "Eth":
            eth = thing
            self.eth += eth
    
    def __iadd__(self, thing):
        self.__add__(thing)
    
    def __isub__(self, thing):
        self.__sub__(thing)
    
    def lookupNftOwner(self, nft):
        return Contract.lookupNftOwner(nft)
    
    ownerOf = lookupNftOwner
    
    def __sub__(self, thing):
        if type(thing).__name__ == 'Nft':
            nft = thing
            self.burn(nft)
        elif type(thing).__name__ == "Eth":
            eth = thing
            self.eth -= eth
    
    def __contains__(self, asset):
        return asset in self.assets
    
    def __repr__(self):
        return f"Owner({self.address},{self.assets})"
    
    def __str__(self):
        return f"Owner({self.address})"
    
    def emit(self, *args):
        self.eventLogger.emit(*args)
    
    def approve(self, to, nft):
        self.emit('Approval',self.address,to.address,nft.contractType,nft.contractName)
        Contract.lookupNftContract(nft).approve(to.address, nft.nftAddr, self.address)
    
    def transferFrom(self,addrFrom,to,nft):
        self.emit('Transfer', addrFrom.address, to.address, nft.nftAddr)
        Contract.lookupNftContract(nft).transferFrom(addrFrom.address,to.address,nft.nftAddr,self.address)
    
    def setInitialOwner(self, to, nft):
        self.emit('SetInitialOwner',to.address,nft.nftAddr,nft.contractType,nft.contractName)
        #Contract.lookupNftContract(nft).setInitialOwner(to.address,nft.nftAddr,self.address)
        Contract.contractDict()['ERC721'][0].setInitialOwner(to.address,nft.nftAddr,self.address)
    
    def setInitialOwners(self, toList, nfts):
        [self.setInitialOwner(theTo, theNft) for theTo,theNft in zip(toList,nfts)]
    
    def burn(self,nft):
        self.emit('Burn',nft.nftAddr,nft.contractType,nft.contractName)
        Contract.lookupNftContract(nft)._burn(nft.nftAddr,self.address)
    
    def N3RP(self, lenderAddr: Address, borrowerAddr: Address, nftAddr: Address, basePayment: Eth, collateral: Eth, rentalDueDate: str, collateralPayoutPeriod: str):
        if 'N3RPCONTRACTBANK' not in globals().keys():
            globals()['N3RPCONTRACTBANK'] = {}
        currNumOfN3RPContracts = len(globals()["N3RPCONTRACTBANK"])
        globals()[f'N3RPNUMBER{currNumOfN3RPContracts}'] = N3RP(lenderAddr, borrowerAddr, self.address, nftAddr, basePayment, collateral, rentalDueDate, collateralPayoutPeriod)
        globals()['N3RPCONTRACTBANK'][f'N3RPNUMBER{currNumOfN3RPContracts}'] = globals()[f'N3RPNUMBER{currNumOfN3RPContracts}']
    
    def depositNft(self, N3RPContract: N3RP):
        N3RPContract.depositNft(N3RPContract.nftAddr, self.address)
        self.emit('DepositNft',self,N3RPContract.nftAddr,N3RPContract.nftContractType,N3RPContract.nftContractName)
        self.transferFrom(N3RPContract.lenderAddr, N3RPContract.address, N3RPContract.nft)
    
    def depositEth(self, N3RPContract: N3RP):
        assert self.eth > N3RPContract.basePayment + N3RPContract.collateral, f"Eth Deposit Error: your {self.eth} is insufficient for this contract requiring {N3RP.basePayment + N3RP.collateral}"
        self.eth -= N3RPContract.basePayment + N3RPContract.collateral
        self.emit('DepositEth',self,N3RPContract.nftAddr,N3RPContract.nftContractType,N3RPContract.nftContractName)
        N3RPContract.depositEth(N3RPContract.basePayment, N3RPContract.collateral, self.address)
    
    @staticmethod
    def contractTypes():
        return Contract.contractTypes()
    
    @staticmethod
    def contractDict():
        return Contract.contractDict()


class EventLogger:
    def __init__(self):
        self.log = {}
        self.activeClasses = []
    
    def __iadd__(self, c):
        if str(c) not in self.activeClasses:
            self.activeClasses.append(str(c))
    
    @property
    def events(self):
        return list(self.log.keys())
    
    def emit(self,eventName,eventCaller,eventResponder,*args):
        currTime = datetime.now().strftime('%Y/%M/%d %H:%M:%S')
        result = [eventName, eventCaller, eventResponder, currTime, *args]
        if eventName not in self.log.keys():
            self.log[eventName] = [result]
        else:
            self.log[eventName] += [result]
    
    def __repr__(self):
        return f"EventLogger({self.log},{self.activeClasses})"

class Contract:
    def __init__(self,contractTypeName,contract):
        if 'LISTOFALLCONTRACTTYPES' not in globals():
            globals()['LISTOFALLCONTRACTTYPES'] = [contractTypeName]
        else:
            globals()['LISTOFALLCONTRACTTYPES'] += [contractTypeName]
        if 'DICTIONARYOFALLCONTRACTS' not in globals():
            globals()['DICTIONARYOFALLCONTRACTS'] = {}
        if contractTypeName not in globals()['DICTIONARYOFALLCONTRACTS'].keys():
            globals()['DICTIONARYOFALLCONTRACTS'][contractTypeName] = [contract]
        if contract not in globals()['DICTIONARYOFALLCONTRACTS'][contractTypeName]:
            globals()['DICTIONARYOFALLCONTRACTS'][contractTypeName] += [contract]
    
    @staticmethod
    def contractTypes():
        return globals()['LISTOFALLCONTRACTTYPES']
    
    @staticmethod
    def contractDict():
        return globals()['DICTIONARYOFALLCONTRACTS']
    
    @staticmethod
    def nftsOfAddress(address: Address):
        return [[el[0] for el in filter(lambda x: x[1] == address, _._owners.items())] for _ in Contract.contractDict()['ERC721']][0]
    
    @staticmethod
    def lookupNftOwner(nft: Nft):
        return [_.ownerOf(nft.nftAddr) for _ in Contract.contractDict()['ERC721']][0]
    
    ownerOf = lookupNftOwner
    
    @staticmethod
    def lookupNftContract(nft: Nft):
        contractList = [_ for _ in Contract.contractDict()['ERC721'] if nft.nftAddr in _._owners.keys()]
        if len(contractList) == 1:
            return contractList[0]
        return None


class ERC721(Contract):
    def __init__(self, name_: Address, symbol_: str):
        self._name = name_
        self._symbol = symbol_
        self._owners = {}
        self._balances = {}
        self._tokenApprovals = {}
        self._operatorApprovals = {}
        self.eventLogger = EventLogger()
        self.allTokensAssigned = False
        self.nftsRemainingToAssign = 10000
        super().__init__(type(self).__name__, self)
    
    @property
    def log(self):
        return self.eventLogger.log
    
    def __repr__(self):
        return f"ERC721({self._name}, {self._symbol}, {self._owners}, {self._balances}, {self._tokenApprovals}, {self._operatorApprovals}, {self.eventLogger}, {self.allTokensAssigned})"
    
    def __str__(self):
        return f"ERC721({self._name})"
    
    def _exists(self, nftAddr):
        return nftAddr in self._owners.keys()
    
    def balanceOf(self, ownerAddress):
        if ownerAddress in self._balances.keys():
            return self._balances[ownerAddress]
        return None
    
    def ownerOf(self, nftAddr):
        if self._exists(nftAddr):
            return self._owners[nftAddr]
        return None
    
    def _approve(self, to, nftAddr):
        self._tokenApprovals[nftAddr] = to
        self.emit("Approval",self.ownerOf(nftAddr), to, nftAddr)
    
    def approve(self, to, nftAddr, msgSender):
        owner = self.ownerOf(nftAddr)
        assert owner != to, "ERC721: approve caller is not owner nor approved for all"
        #assert msgSender == owner | isApprovedForAll(owner, msgSender), "Error"
        self._approve(to, nftAddr)
    
    def isApprovedForAll(self,owner,operator):
        if owner in self._operatorApprovals.keys():
            return operator in self._operatorApprovals[owner].keys()
        return False
    
    def transferFrom(self,addrFrom,to,nftAddr,msgSender):
        assert self._isApprovedOrOwner(msgSender, nftAddr), "ERC721: transfer caller is not owner nor approved"
        self._transfer(addrFrom, to, nftAddr)
    
    def _isApprovedOrOwner(self, spender, nftAddr):
        assert self._exists(nftAddr), "ERC721: operator query for nonexistent token"
        owner = self.ownerOf(nftAddr)
        print(f'{spender=}, {nftAddr=}, {owner=}')
        print((spender == owner), (self.getApproved(nftAddr) == spender))
        return (spender == owner) | (self.getApproved(nftAddr) == spender) | (self.isApprovedForAll(owner, spender))
    
    def getApproved(self, nftAddr):
        assert self._exists(nftAddr), "ERC721: approved query for nonexistent token"
        return nftAddr in self._tokenApprovals.keys();
    
    def _transfer(self, addrFrom, to, nftAddr):
        assert self.ownerOf(nftAddr) == addrFrom, "ERC721: transfer from incorrect owner"
        assert to != None, "ERC721: transfer to the zero address"
        self._beforeTokenTransfer(addrFrom, to, nftAddr)
        self._approve(None, nftAddr)
        self._balances[addrFrom] -= 1
        if to not in self._balances.keys():
            self._balances[to] = 0
        self._balances[to] += 1
        self._owners[nftAddr] = to
        #self.eventLogger.emit("Transfer",addrFrom, to, nftAddr)
        self.emit("Transfer",addrFrom, to, nftAddr)
        self._afterTokenTransfer(addrFrom, to, nftAddr)
    
    def _afterTokenTransfer(self, addrFrom, to, nftAddr):
        pass
    
    def _safeMint(self, to, nftAddr, _data=""):
        self._mint(to, nftAddr)
        assert self._checkOnERC721Received(None, to, nftAddr, _data), "ERC721: transfer to non ERC721Receiver implementer"
    
    def _checkOnERC721Received(self, addrFrom, to, nftAddr, _data):
        #Add in later
        return True
    
    def _mint(self, to, nftAddr):
        assert to != None, "ERC721: mint to the zero address"
        assert not self._exists(nftAddr), "ERC721: token already minted"
        self._beforeTokenTransfer(None, to, nftAddr)
        if to not in self._balances.keys():
            self._balances[to] = 0
        self._balances[to] += 1
        self._owners[nftAddr] = to
        #self.eventLogger.emit("Transfer",None, to, nftAddr)
        self.emit("Transfer",None, to, nftAddr)
        self._afterTokenTransfer(None, to, nftAddr)
    
    def _beforeTokenTransfer(self, addrFrom, to, nftAddr):
        pass
    
    def _afterTokenTransfer(self, addrFrom, to, nftAddr):
        pass
    
    def setInitialOwner(self, to, nftAddr, msgSender):
        self.emit("Mint",msgSender,to,nftAddr)
        self._mint(to, nftAddr)
    
    def setInitialOwners(self, toList, nftAddrs, msgSender):
        [self.setInitialOwner(to, nftAddr, msgSender) for to,nftAddr in zip(toList,nftAddrs)]
    
    def emit(self,*args):
        self.eventLogger.emit(*args)
    
    def _burn(self, nftAddr, msgSender):
        owner = self.ownerOf(nftAddr)
        self._beforeTokenTransfer(owner, None, nftAddr)
        
        self._approve(None, nftAddr)
        
        self._balances[owner] -= 1
        del self._owners[nftAddr]
        
        self.emit("Transfer", msgSender, owner, None, nftAddr)
        
        self._afterTokenTransfer(owner, None, nftAddr)


class N3RP(Contract):
    def __init__(self,
                 lenderAddr: Address,
                 borrowerAddr: Address,
                 initiatorAddr: Address,
                 nft: Nft,
                 basePayment: Eth,
                 collateral: Eth,
                 rentalDueDate: str, #Will be a ethereum blocktime, in reality
                 collateralPayoutPeriod: str, #Will be an integer number of blocks, in reality
                ):
        self.address = Address(type(self).__name__) + nft.nftAddr
        self.eth = Eth(0)
        self.str = self.address
        self.lenderAddr = lenderAddr
        self.borrowerAddr = borrowerAddr
        self.initiatorAddr = initiatorAddr
        self.nft = nft
        self.basePayment = basePayment
        self.collateral = collateral
        self.rentalDueDate = rentalDueDate
        self.collateralPayoutPeriod = collateralPayoutPeriod
        super().__init__(type(self).__name__,self)
    
    @property
    def nftAddr(self):
        return self.nft.nftAddr
    
    @property
    def nftContractName(self):
        return self.nft.contractName
    
    @property
    def nftContractType(self):
        return self.nft.contractType
    
    @property
    def nftIsDeposited(self):
        return self.address == self.lookupNftOwner(self.nft)
    
    @property
    def ethIsDeposited(self):
        return self.basePayment + self.collateral >= self.eth
    
    def lookupNftOwner(self, nft):
        return Contract.lookupNftOwner(nft)
    
    ownerOf = lookupNftOwner
    
    def testFunction(self):
        return super().contractDict()
    
    def depositNft(self, nftAddr: Address, msgSender: Address):
        assert self.nftIsDeposited is False, f"Nft Deposit Error: an nft has already been deposited here"
        assert self.lenderAddr == msgSender, f"Nft Deposit Error: sender {msgSender} is not lender {self.lenderAddr}"
        assert self.nftAddr == nftAddr, f"Nft Deposit Error: deposited nft {nftAddr} does not match the stipulated nft {self.nftAddr}"
        return True
    
    def depositEth(self, basePayment: Eth, collateral: Eth, depositorAddr: Address):
        assert self.ethIsDeposited is False, f"Eth Deposit Error: eth has already been deposited here"
        assert self.basePayment == basePayment, f"Eth Deposit Error: a base payment of {self.basePayment} is required, but {basePayment} was received"
        assert self.collateral == collateral, f"Eth Deposit Error: a collateral of {self.collateral} is required, but {collateral} was received"
        assert self.depositorAddr == depositorAddr, f"Eth Deposit Error: sender {depositorAddr} is not the depositor {self.depositor}"
        self.eth = self.basePayment + self.depositorAddr
        return True
    
    def __repr__(self):
        return f"N3RP({self.address},{self.eth},{self.str},{self.lenderAddr},{self.borrowerAddr},{self.initiatorAddr},{self.nft},{self.basePayment},{self.collateral},{self.rentalDueDate},{self.collateralPayoutPeriod},{self.nftIsDeposited},{self.ethIsDeposited})"


In [3]:
theERC721 = ERC721(Address('CryptoPunks'), 'CRYPTOPUNKS')

nft1 = Nft(Address('JjCoin'), 1, Eth(100), '', theERC721)
nft2 = Nft(Address('GgCoin'), 2, Eth(1000), '', theERC721)
owner1 = Owner('Gg')
owner2 = Owner('Jj')
owner3 = Owner('xXxCryptoLordxXx')
# owner3.approve(owner2.address,nft2.nftId,'CryptoPunks')

In [4]:
lender = Owner(Address('Lender'))
lender.setInitialOwner(lender, nft1)
borrower = Owner(Address('Borrower'))
borrower + Eth(1000)

In [5]:
print(Contract.lookupNftContract(nft1))

ERC721(Address(0xCryptoPunks))


In [6]:
[_ for _ in Contract.contractDict()['ERC721']]

[ERC721(Address(0xCryptoPunks), CRYPTOPUNKS, {Address(0xJjCoin): Address(0xLender)}, {Address(0xLender): 1}, {}, {}, EventLogger({'Mint': [['Mint', Address(0xLender), Address(0xLender), '2022/38/07 00:38:49', Address(0xJjCoin)]], 'Transfer': [['Transfer', None, Address(0xLender), '2022/38/07 00:38:49', Address(0xJjCoin)]]},[]), False)]

In [10]:
Contract.contractDict()['ERC721'][0]._owners

{Address(0xJjCoin): Address(0xLender)}

In [13]:
owner2.approve(owner2.address,nft1)
lender.transferFrom(lender.address,owner2.address,nft1)

spender=Address(0xLender), nftAddr=Address(0xJjCoin), owner=Address(0xLender)
True False


In [15]:
owner2.assets

[[Address(0xJjCoin)], Ξ0]

In [ ]:
# lenderAddr, borrowerAddr, nftAddr, basePayment, collateral, rentalDueDate
lender.N3RP(lender.address, borrower.address, nft1, Eth(2), Eth(25), '20220209', '20220211')

In [ ]:
lender

In [ ]:
lender.nfts

In [ ]:
Contract.__dict__

In [ ]:
borrower.lookupNftOwner(nft2)

In [ ]:
lender.setInitialOwner(lender, nft2)

In [ ]:
theERC721._owners

In [ ]:
Contract.contractDict()['ERC721'][0]._owners

In [ ]:
nft3 = Nft(Address('AsdfCoin'),3,Eth(10000),'',theERC721)

In [ ]:
owner1.setInitialOwner(owner1,nft3)

In [ ]:
borrower.lookupNftOwner(nft3)

In [ ]:
fdsa = Eth(100)
fdsa += Eth(50)
fdsa + Eth(2)

In [ ]:
[np.array(list(filter(lambda x: x[1] == lender.address, _._owners.items())))[:][1] for _ in Contract.contractDict()['ERC721']][0]
# [_._owners for _ in Contract.contractDict()['ERC721']]

In [ ]:
Contract.contractDict()['ERC721'][0]._owners

In [ ]:
[_.ownerOf(nft1.nftAddr) for _ in Contract.contractDict()['ERC721']][0]

In [ ]:
[[el[0] for el in filter(lambda x: x[1] == lender.address, _._owners.items())] for _ in Contract.contractDict()['ERC721']][0]